# Upload synthetics to dandi

author: laquitainesteeve@gmail.com

Execution time: 10 min

Special hardware: on CPU, does not require GPU.

# Setup 

Activate demo virtual environment (envs/demo.yml)

```bash
python -m ipykernel install --user --name demo --display-name "demo"
```

In [14]:
%%time 

# import python packages
import os
import yaml
import numpy as np
from time import time
from dandi.dandiapi import DandiAPIClient
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface
import spikeinterface as si
from pynwb.file import NWBFile, Subject
from pynwb import NWBHDF5IO
import uuid
from datetime import datetime
from dateutil.tz import tzlocal
from neuroconv.tools.spikeinterface import add_recording_to_nwbfile, add_sorting_to_nwbfile
print("spikeinterface", spikeinterface.__version__)

# setup project
proj_path = "/home/steeve/steeve/epfl/code/spikebias/"
os.chdir(proj_path)

from src.nodes.dandi import write_nwb

# setup pipeline parameters
DANDISET_ID = '001250'

# setup nwb files save paths
FILE_PATH = os.path.join(proj_path, "temp/pros_of_details/")


spikeinterface 0.101.2
CPU times: user 535 μs, sys: 0 ns, total: 535 μs
Wall time: 898 μs


## Upload buccino replicate

In [15]:
%%time 

# setup parameters
SUBJECT_ID = "mearec-synthetics"
SESSION_ID = 'buccino_replicate'
SESSION_DESCRIPTION = 'Replication of Buccino synthetic model'        
RECORDING_PATH = 'dataset/00_raw/recording_buccino_rep/'
GROUND_TRUTH_PATH = 'dataset/00_raw/ground_truth_buccino_rep/'
SUBJECT_PATH = os.path.join(FILE_PATH, f"{DANDISET_ID}/sub-{SUBJECT_ID}/")
NWB_FILE_PATH = os.path.join(SUBJECT_PATH, f"sub-{SUBJECT_ID}_ses-{SESSION_ID}.nwb")
PARAMS = {
    "subject_id": SUBJECT_ID,
    "session_description": SESSION_DESCRIPTION,
    "identifier": str(uuid.uuid4()),
    "session_start_time": datetime.now(tzlocal()),
    "experimenter": "Laquitaine Steeve",
    "lab": "Blue Brain Project",
    "institution": "EPFL",
    "experiment_description": SESSION_DESCRIPTION,
    "session_id": SESSION_ID,
    "related_publications": "doi:"
    }

# load extractors
RecordingS1 = si.load_extractor(RECORDING_PATH)
SortingGtS1 = si.load_extractor(GROUND_TRUTH_PATH)
    
# write as nwb
write_nwb.run(RECORDING_PATH, GROUND_TRUTH_PATH, NWB_FILE_PATH, PARAMS)

# download a minimal local dandiset with only the .yaml config
os.system(f"dandi download --download dandiset.yaml --output-dir {FILE_PATH} DANDI:001250")

# go to path of the dandiset
os.chdir(FILE_PATH)

# upload
os.system(
    f"""
    export DANDI_API_KEY='210e68743286d64e84743bd8980d5771ef82bf4d';
    cd {os.path.join(FILE_PATH, DANDISET_ID)};
    dandi organize {SUBJECT_PATH} -f dry;
    dandi organize {SUBJECT_PATH};    
    dandi upload
    """
)

# delete local dandiset
os.system(f"rm -rf {os.path.join(FILE_PATH, DANDISET_ID)}")

2025-08-13 14:42:25,054 - root - write_nwb.py - run - INFO - Ground truth metadata:
2025-08-13 14:42:25,055 - root - write_nwb.py - run - INFO - ['z', 'y', 'bursting', 'mtype', 'exp_decay', 'x', 'max_burst_duration', 'max_spikes_per_burst', 'cell_type', 'snr']


/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.5 is recommended
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/neuroconv/tools/spikeinterface/spikeinterface.py:1054: UserWarning: The 'add_recording' function is deprecated and will be removed after February 2025. 
Please use 'add_recording_to_nwbfile' instead.
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/neuroconv/tools/spikeinterface/spikeinterface.py:1676: UserWarning: The 'add_sorting' function is deprecated and will be removed after February 2025. 
Please use 'add_sorting_to_nwbfile' instead.
  warnings.warn(


2025-08-13 14:42:25,324 - root - utils.py - create_if_not_exists - INFO - The following path has been created /home/steeve/steeve/epfl/code/spikebias/temp/pros_of_details/001250/sub-mearec-synthetics
PATH                 SIZE DONE    DONE% CHECKSUM STATUS MESSAGE   
001250/dandiset.yaml                             done   updated   
Summary:                  0 Bytes                1 done 1 updated 
                          <0.00%                                  


2025-08-13 14:42:27,998 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.08.13-12.42.27Z-3389758.log
2025-08-13 14:42:28,792 [    INFO] Loading metadata from 1 files
2025-08-13 14:42:28,911 [    INFO] Organized 0 out of 1 paths. Visit /home/steeve/steeve/epfl/code/spikebias/temp/pros_of_details/001250/
2025-08-13 14:42:28,911 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.08.13-12.42.28Z-3389768.log


DRY: /home/steeve/steeve/epfl/code/spikebias/temp/pros_of_details/001250/sub-mearec-synthetics/sub-mearec-synthetics_ses-buccino_replicate.nwb -> sub-mearec-synthetics/sub-mearec-synthetics_ecephys.nwb


2025-08-13 14:42:29,751 [    INFO] Loading metadata from 1 files
2025-08-13 14:42:29,753 [    INFO] Symlink support autodetected; setting files_mode='symlink'
2025-08-13 14:42:29,756 [    INFO] Organized 1 paths. Visit /home/steeve/steeve/epfl/code/spikebias/temp/pros_of_details/001250/
2025-08-13 14:42:29,756 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.08.13-12.42.29Z-3389847.log
2025-08-13 14:42:31,342 [    INFO] Found 3 files to consider


PATH                                                                  SIZE    ERRORS  PROGRESS STATUS                MESSAGE                                                                            
dandiset.yaml                                                         5.6 kB                   skipped               should be edited online                                                            
sub-mearec-synthetics/sub-mearec-synthetics_ecephys.nwb               3.0 GB    0         100% done                                                                                                     
sub-mearec-synthetics/sub-mearec-synthetics_ses-buccino_replicate.nwb 3.0 GB    0         100% ERROR                 Error 409 while sending POST request to https://api.dandiarchive.org/api/uploads...
Summary:                                                              6.0 GB         16.8 MB/s 1 skipped             1 should be edited online                                                      

2025-08-13 14:48:29,258 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.08.13-12.42.30Z-3389920.log
Error: Error 409 while sending POST request to https://api.dandiarchive.org/api/uploads/26e47b78-fe37-43a0-bd34-4862cc2812ef/validate/: "An identical blob has already been uploaded."


CPU times: user 1.01 s, sys: 822 ms, total: 1.83 s
Wall time: 6min 4s


0